# Nền tảng của UnifiedH97ResNet

Có 1 khung tạo 3 model như sau: 

```python
import torch
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary

class H97_ResNet(nn.Module):
    def __init__(self, num_classes: int = 3):
        super(H97_ResNet, self).__init__()        
        # Tải mô hình ResNet50 đã được huấn luyện trước
        resnet50 = models.resnet50(pretrained=True)
        # Loại bỏ lớp fully connected cuối cùng
        self.feature_extractor = nn.Sequential(*list(resnet50.children())[:-1])
        # Đóng băng các tham số trong feature extractor để không cập nhật trong quá trình huấn luyện
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

        # Kích thước đầu vào cho lớp fully connected đầu tiên dựa trên output của ResNet50
        # ResNet50 thường trả về tensor [batch_size, 2048, 1, 1] sau lớp pooling cuối cùng
        self.fc1 = nn.Linear(2048, 9)
        self.fc2 = nn.Linear(9, 7)
        self.fc3 = nn.Linear(7, 3)
        self.fc4 = nn.Linear(3, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # Trích xuất đặc trưng
        x = self.feature_extractor(x)
        # Chuyển đổi tensor từ [batch_size, 2048, 1, 1] sang [batch_size, 2048] để phù hợp với lớp fully connected
        x = torch.flatten(x, 1)
        # Đưa qua mạng dense
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)

        x = self.fc4(x)
        return x
```

3 model lần lượt là:
- h97_ResNet_B2_B5B6 (M1): phân biệt nhãn B2 với 2 nhãn còn lại B5B6 = H97_ResNet(num_classes=2)
- h97_ResNet_B5_B2B6 (M2): phân biệt nhãn B5 với 2 nhãn còn lại B2B6 = H97_ResNet(num_classes=2)
- h97_ResNet_B6_B2B5 (M3): phân biệt nhãn B6 với 2 nhãn còn lại B2B5 = H97_ResNet(num_classes=2)
- h97_ResNet_B2_B5_B6 (M4): phân biệt các nhãn B2, B5, B6 với nhau = H97_ResNet(num_classes=3)

Vì cả 4 model đều có chung phần ResNet50 giữ nguyên không đổi trọng số nên tôi đang nghĩ đến việc hợp nhất 4 model.
Cụ thể: 
M1 qua 3 tầng fc (9, 7, 3) cho ra 2 output
M2 qua 3 tầng fc (9, 7, 3) cho ra 2 output
M3 qua 3 tầng fc (9, 7, 3) cho ra 2 output
M4 qua 3 tầng fc (9, 7, 3) cho ra 3 output
Việc dùng chung ResNet vào sẽ giúp giảm khối lượng mô hình. Bên cạnh đó với 9 đầu ra của các model (2 + 2 + 2 + 3) từ M1, M2, M3, M4 có thể dùng thêm 1 lớp fc (3) hàm kích hoạt softmax cho phân loại 3 nhãn B2, B5, B6.

```markdown
Kiến trúc thu được sẽ giống như sau:
            ___fc (9, 7, 3, 2) của M1 ra 2 đặc trưng____
            |__fc (9, 7, 3, 2) của M2 ra 2 đặc trưng___|
ResNet50----|__fc (9, 7, 3, 2) của M3 ra 2 đặc trưng___|___fc(3) tương ứng 3 đặc trưng phân loại các nhãn B2, B5, B6.
            |__fc (9, 7, 3, 3) của M4 ra 3 đặc trưng___|
```

Hãy viết model triển khai kiến trúc như trên.

# Script load trọng số 4 model đã train vòa UnifiedModel và lưu lại thiết lập của UnifiedModel

B1. Load UnifiedModel trọng số các lớp của 4 model đã train

B2. Save lại model với trọng số tại: /path/to/UnifiedModel.pt

B3. Sửa lại hàm dựng để nó tự động load lại trọng số trong file /path/to/UnifiedModel.pt

In [4]:
from UnifiedH97ResNet import UnifiedModel
from H97 import H97_ResNet
from torch import load, save, device

B2_B5B6_dir = '/mnt/Data/Projects/h97_resnet_B2_B5B6_dataver1/best_h97_resnet_B2_B5B6_dataver1_model.pt'
B5_B2B6_dir = '/mnt/Data/Projects/h97_resnet_B5_B2B6_dataver1/best_h97_resnet_B5_B2B6_dataver1_model.pt'
B6_B2B5_dir = '/mnt/Data/Projects/h97_resnet_B6_B2B5_dataver1/best_h97_resnet_B6_B2B5_dataver1_model.pt'
B2_B5_B6_dir = '/mnt/Data/Projects/h97_resnet_B2_B6_B5_dataver1/best_h97_resnet_B2_B6_B5_dataver1_model.pt'

models_dir = [B2_B5B6_dir, B5_B2B6_dir, B6_B2B5_dir, B2_B5_B6_dir]

In [5]:
unified_model = UnifiedModel()

In [7]:
# Load model B2_B5B6_dir
sub_model = H97_ResNet(num_classes=2)
sub_model.load_state_dict(load(B2_B5B6_dir, map_location=device('cpu')))
unified_model.fc1_B2_B5B6 = sub_model.fc1
unified_model.fc2_B2_B5B6 = sub_model.fc2
unified_model.fc3_B2_B5B6 = sub_model.fc3
unified_model.fc4_B2_B5B6 = sub_model.fc4

/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harito/venv/pyvenv311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Load model B5_B2B6_dir
sub_model = H97_ResNet(num_classes=2)
sub_model.load_state_dict(load(B5_B2B6_dir, map_location=device('cpu')))
unified_model.fc1_B5_B2B6 = sub_model.fc1
unified_model.fc2_B5_B2B6 = sub_model.fc2
unified_model.fc3_B5_B2B6 = sub_model.fc3
unified_model.fc4_B5_B2B6 = sub_model.fc4

In [9]:
# Load model B6_B2B5_dir
sub_model = H97_ResNet(num_classes=2)
sub_model.load_state_dict(load(B6_B2B5_dir, map_location=device('cpu')))
unified_model.fc1_B6_B2B5 = sub_model.fc1
unified_model.fc2_B6_B2B5 = sub_model.fc2
unified_model.fc3_B6_B2B5 = sub_model.fc3
unified_model.fc4_B6_B2B5 = sub_model.fc4

In [10]:
# Load model B2_B5_B6_dir
sub_model = H97_ResNet(num_classes=3)
sub_model.load_state_dict(load(B2_B5_B6_dir, map_location=device('cpu')))
unified_model.fc1_B2_B5_B6 = sub_model.fc1
unified_model.fc2_B2_B5_B6 = sub_model.fc2
unified_model.fc3_B2_B5_B6 = sub_model.fc3
unified_model.fc4_B2_B5_B6 = sub_model.fc4

In [11]:
save(unified_model.state_dict(), 'unified_model_weights.pt')

# Script chuẩn bị data và huấn luyện model